In [1]:
import math
from scipy.io import arff
from scipy.stats.stats import pearsonr
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

# Formatação mais bonita para os notebooks
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15,5)


# IEEE

In [99]:
df_IEEE = pd.read_csv('data/ieee_sistematic_research_data.csv',  header=0)
df_IEEE.head()

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication_Year,Volume,Issue,Start Page,End Page,...,INSPEC Non-Controlled Terms,Mesh_Terms,Article Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,A 95pJ/label Wide-Range Depth-Estimation Proce...,L. Chen; Y. Lu; Y. Hiao; B. Yang; W. Chen; C. ...,"Department of Electrical Engineering, National...",2018 IEEE Asian Solid-State Circuits Conferenc...,NaN,2018,NaN,NaN,261,262,...,octave search range sampling;FPGA;digital refo...,NaN,NaN,4,NaN,NaN,NaN,NaN,IEEE,IEEE Conferences
1,An efficient FPGA implementation of HEVC intra...,H. Azgin; A. C. Mert; E. Kalali; I. Hamzaoglu,"Faculty of Engineering and Natural Sciences, S...",2018 IEEE International Conference on Consumer...,NaN,2018,NaN,NaN,1,5,...,High Efficiency Video Coding standard;HEVC int...,NaN,3.0,16,NaN,NaN,NaN,NaN,IEEE,IEEE Conferences
2,FPGA-Based Real-Time Super-Resolution System f...,Z. He; H. Huang; M. Jiang; Y. Bai; G. Luo,NA; NA; NA; NA; NA,2018 IEEE 26th Annual International Symposium ...,NaN,2018,NaN,NaN,181,188,...,FPGA-based real-time super-resolution system;F...,NaN,NaN,47,NaN,NaN,NaN,NaN,IEEE,IEEE Conferences
3,Full-HD Accelerated and Embedded Feature Detec...,L. Kalms; H. Ibrahim; D. Göhringer,"Technische Universitát Dresden, Dresden, Germa...",2018 International Conference on ReConFigurabl...,NaN,2018,NaN,NaN,1,6,...,power constraints;clock-accurate computations;...,NaN,NaN,17,NaN,NaN,NaN,NaN,IEEE,IEEE Conferences
4,Design of a Gabor Filter HW Accelerator for Ap...,G. D. Licciardo; C. Cappetta; L. Di Benedetto,"Department of Industrial Engineering, Universi...","IEEE Transactions on Components, Packaging and...",NaN,2018,8.0,7.0,1187,1194,...,Gabor filter HW accelerator;medical imaging;GF...,NaN,NaN,37,NaN,NaN,NaN,NaN,IEEE,IEEE Journals & Magazines


In [100]:
df_resource = df_IEEE[['Document Title','Abstract', 'Publication_Year', 'PDF Link']]
df_resource
# df_resource.shape


,Document Title,Abstract,Publication_Year,PDF Link
0,A 95pJ/label Wide-Range Depth-Estimation Proce...,High-resolution and wide-range depth maps are ...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
1,An efficient FPGA implementation of HEVC intra...,Intra prediction algorithm used in High Effici...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
2,FPGA-Based Real-Time Super-Resolution System f...,The market benefits from a barrage of Ultra Hi...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
3,Full-HD Accelerated and Embedded Feature Detec...,Detecting features using pattern recognition a...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
4,Design of a Gabor Filter HW Accelerator for Ap...,The Gabor filter (GF) has been proved to show ...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
5,Hardware Architecture for Fast General Object ...,"For embedded system applications, high detecti...",2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
6,A Reconfigurable Fractional Interpolation Hard...,Fractional interpolation is one of the most co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
7,A Low Power Versatile Video Coding (VVC) Fract...,Fractional interpolation in Versatile Video Co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
8,Design and Implementation of 2D IDCT/IDST-Spec...,The paper talks about how to implement differe...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
9,Design Criteria for Real-time Processing of HW...,Gabor filters gained a great importance in mul...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...


In [101]:
# df_resource[df_resource['Document Title'].str.contains('decoder|decoding|HEVC|deblocking|accelerator|accelerated|neural network|verified in FPGA|rendering|VLSI design|motion estimation|network on chip', case=False)]['Document Title'].size
# df_resource[df_resource['Abstract'].str.contains('accelerator|accelerated|verified in FPGA|rendering', case=False)==False]['Document Title'].size

df_ieee_cleaned_research = df_resource[((df_resource['Document Title'].str.contains('decoder|decoding|HEVC|deblocking|accelerator|accelerated|neural network|verified in FPGA|rendering|VLSI design|motion estimation|network on chip', case=False)) | (df_resource['Abstract'].str.contains('accelerator|accelerated|verified in FPGA|rendering', case=False))==False)]
df_ieee_cleaned_research.shape


(6, 4)

In [102]:
df_ieee_cleaned_research

,Document Title,Abstract,Publication_Year,PDF Link
0,A 95pJ/label Wide-Range Depth-Estimation Proce...,High-resolution and wide-range depth maps are ...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
2,FPGA-Based Real-Time Super-Resolution System f...,The market benefits from a barrage of Ultra Hi...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
6,A Reconfigurable Fractional Interpolation Hard...,Fractional interpolation is one of the most co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
7,A Low Power Versatile Video Coding (VVC) Fract...,Fractional interpolation in Versatile Video Co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
9,Design Criteria for Real-time Processing of HW...,Gabor filters gained a great importance in mul...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...
10,High-Speed CMOS Color Stereo Camera with Low C...,The paper presents alow complexity hardware ar...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...


In [104]:
publish_months = [11,5,8,10,5,6]
df_ieee_cleaned_research['Publication_month'] = publish_months

df_ieee_cleaned_research = df_ieee_cleaned_research[(df_ieee_cleaned_research['Publication_Year'] > 2017) & (df_ieee_cleaned_research['Publication_month'] > 6)]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [105]:
df_ieee_cleaned_research

,Document Title,Abstract,Publication_Year,PDF Link,Publication_month
0,A 95pJ/label Wide-Range Depth-Estimation Proce...,High-resolution and wide-range depth maps are ...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,11
6,A Reconfigurable Fractional Interpolation Hard...,Fractional interpolation is one of the most co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,8
7,A Low Power Versatile Video Coding (VVC) Fract...,Fractional interpolation in Versatile Video Co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,10


In [107]:
df_cleaned_research.to_csv('data/ieee_cleaned_results.csv', index=False)
df_cleaned_research

,Document Title,Abstract,Publication_Year,PDF Link,Publication_month
0,A 95pJ/label Wide-Range Depth-Estimation Proce...,High-resolution and wide-range depth maps are ...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,11
6,A Reconfigurable Fractional Interpolation Hard...,Fractional interpolation is one of the most co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,8
7,A Low Power Versatile Video Coding (VVC) Fract...,Fractional interpolation in Versatile Video Co...,2018,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,10


# Science Direct

In [92]:
df_science_direct = pd.read_csv('data/science_direct_sistematic_results.csv',  header=0)
df_science_direct.head()

,Document Title,Abstract,Publication_month,Publication_Year,PDF Link
0,Real-time Frame Buffer Implementation based on...,"In this paper, design of a real-time video fra...",5,2018,https://www.sciencedirect.com/science/article/...
1,Real-time HDTV to 4K and 8K-UHD conversions us...,The demand for light-weight and high-speed sup...,9,2018,https://www.sciencedirect.com/science/article/...
2,System-on-a-chip (SoC)-based hardware accelera...,The rapid growth of embedded vision applicatio...,3,2018,https://www.sciencedirect.com/science/article/...
3,Fast adaptive motion estimation algorithm and ...,Motion estimation (ME) plays an important part...,2,2018,https://www.sciencedirect.com/science/article/...
4,A Survey Of mobile face biometrics,Face biometrics have attracted significant att...,8,2018,https://www.sciencedirect.com/science/article/...


In [93]:
df_science_direct_cleaned = df_SD[((df_SD['Document Title'].str.contains('decoder|decoding|HEVC|deblocking|accelerator|accelerated|neural network|verified in FPGA|rendering|VLSI design|motion estimation|network on chip', case=False)) | (df_SD['Abstract'].str.contains('accelerator|accelerated|verified in FPGA|rendering', case=False))==False)]
df_science_direct_cleaned.shape


(6, 5)

In [94]:
df_science_direct_cleaned

,Document Title,Abstract,Publication_month,Publication_Year,PDF Link
0,Real-time Frame Buffer Implementation based on...,"In this paper, design of a real-time video fra...",5,2018,https://www.sciencedirect.com/science/article/...
1,Real-time HDTV to 4K and 8K-UHD conversions us...,The demand for light-weight and high-speed sup...,9,2018,https://www.sciencedirect.com/science/article/...
2,System-on-a-chip (SoC)-based hardware accelera...,The rapid growth of embedded vision applicatio...,3,2018,https://www.sciencedirect.com/science/article/...
4,A Survey Of mobile face biometrics,Face biometrics have attracted significant att...,8,2018,https://www.sciencedirect.com/science/article/...
6,A novel GPU-aware Histogram-based algorithm fo...,Multimedia data are a popular case of Big Data...,3,2019,https://www.sciencedirect.com/science/article/...
8,Signal processing challenges for digital holog...,Holography is considered to be the ultimate di...,2,2019,https://www.sciencedirect.com/science/article/...


In [97]:
df_science_direct_cleaned = df_science_direct_cleaned[(df_science_direct_cleaned['Publication_Year'] > 2017) & (df_science_direct_cleaned['Publication_month'] > 6)]
df_science_direct_cleaned

,Document Title,Abstract,Publication_month,Publication_Year,PDF Link
1,Real-time HDTV to 4K and 8K-UHD conversions us...,The demand for light-weight and high-speed sup...,9,2018,https://www.sciencedirect.com/science/article/...
4,A Survey Of mobile face biometrics,Face biometrics have attracted significant att...,8,2018,https://www.sciencedirect.com/science/article/...


In [98]:
df_science_direct_cleaned.to_csv('data/science_direct_cleaned_results.csv', index=False)
df_science_direct_cleaned

,Document Title,Abstract,Publication_month,Publication_Year,PDF Link
1,Real-time HDTV to 4K and 8K-UHD conversions us...,The demand for light-weight and high-speed sup...,9,2018,https://www.sciencedirect.com/science/article/...
4,A Survey Of mobile face biometrics,Face biometrics have attracted significant att...,8,2018,https://www.sciencedirect.com/science/article/...


# Google Scholar

In [65]:
df_google_scholar = pd.read_csv('data/google_scholar_sistematic_results.csv',  header=0)
df_google_scholar.shape
# df_google_scholar

(32, 5)

In [67]:
df_google_scholar_cleaned = df_SD[((df_SD['Document Title'].str.contains('decoder|decoding|HEVC|deblocking|accelerator|accelerated|neural network|verified in FPGA|rendering|VLSI design|motion estimation|network on chip', case=False)) | (df_SD['Abstract'].str.contains('accelerator|accelerated|verified in FPGA|rendering', case=False))==False)]
df_google_scholar_cleaned.shape
# df_google_scholar_cleaned


(6, 5)

In [69]:
df_google_scholar_cleaned = df_google_scholar_cleaned[(df_google_scholar_cleaned['Publication_Year'] > 2017) & (df_google_scholar_cleaned['Publication_month'] > 6)]
df_google_scholar_cleaned.to_csv('data/google_scholar_cleaned_results.csv', index=False)
df_google_scholar_cleaned


,Document Title,Abstract,Publication_month,Publication_Year,PDF Link
1,Real-time HDTV to 4K and 8K-UHD conversions us...,The demand for light-weight and high-speed sup...,9,2018,https://www.sciencedirect.com/science/article/...
4,A Survey Of mobile face biometrics,Face biometrics have attracted significant att...,8,2018,https://www.sciencedirect.com/science/article/...
